In [2]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Okt
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora 
from gensim import models
import gensim

In [3]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [4]:
data =[]
for i in range(0,1):
    txt1 = open('문종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '문종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '백성' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'문종실록'))

In [5]:
# 시리즈 만들기
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("문종실록"," "))
df=df.map(lambda x : x.replace("문종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [6]:
len(df)

317

In [7]:
okt = Okt()

In [8]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [9]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [10]:
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [11]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	백성	0.014152348972856998
	하니	0.013632213696837425
	모두	0.008033135905861855
	의논	0.007555556017905474
	만약	0.006741966120898724
	경태	0.0065043107606470585
	고을	0.004996372386813164
	생각	0.004953319206833839
	하소	0.004670554772019386
	때문	0.004541595932096243


Topic ID: 1
	백성	0.012677239254117012
	모두	0.012237949296832085
	하니	0.011817765422165394
	경태	0.007682453375309706
	하소	0.006468811072409153
	고을	0.006336240563541651
	의논	0.006113214418292046
	비록	0.0061053805984556675
	수령	0.005384552292525768
	평안도	0.005275217350572348


Topic ID: 2
	백성	0.010662621818482876
	모두	0.008325895294547081
	비록	0.005819622427225113
	마음	0.005530654918402433
	어찌	0.005455477628856897
	만약	0.005442019551992416
	생각	0.004981666803359985
	다시	0.004342825151979923
	수령	0.004114017356187105
	경태	0.004088385496288538


Topic ID: 3
	폐단	0.012448890134692192
	백성	0.009965571574866772
	비록	0.009814072400331497
	하니	0.009477952495217323
	고을	0.009389382787048817
	어찌	0.0065420386381447315
	만약	0.006464463192

In [12]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.004718 -0.015622       1        1  18.903728
12     0.009563 -0.022387       2        1  10.680546
8     -0.015549  0.010787       3        1   9.196493
19     0.006366 -0.012596       4        1   7.369055
14     0.046300 -0.015229       5        1   6.575198
4     -0.006079 -0.002303       6        1   6.513160
5      0.005981  0.029126       7        1   6.343333
0      0.027732  0.019422       8        1   5.998065
18     0.003853 -0.024394       9        1   4.887814
1      0.037775  0.027650      10        1   4.263629
10    -0.010659 -0.000224      11        1   3.605894
7      0.003991 -0.003221      12        1   2.590187
6     -0.041405  0.020802      13        1   2.468286
17    -0.005539 -0.024221      14        1   2.424529
15     0.019304  0.020087      15        1   1.978816
2     -0.031895  0.009147      16        1   1.466117
16    -0.006961 -0.016918      17        1   1.435877
3     -0.012080 -0.030881      18        1   1.435354
13    -0.018517  0.018269      19        1   1.092536
9     -0.007461  0.012706      20        1   0.771372, topic_info=    Term        Freq       Total Category  logprob  loglift
29    백성  678.000000  678.000000  Default  30.0000  30.0000
21    모두  465.000000  465.000000  Default  29.0000  29.0000
67    하니  571.000000  571.000000  Default  28.0000  28.0000
371   폐단  265.000000  265.000000  Default  27.0000  27.0000
189   비록  360.000000  360.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
529   우리    1.299099  186.761307  Topic20  -5.7699  -0.1034
119   수가    1.427497  270.094513  Topic20  -5.6756  -0.3781
371   폐단    1.387619  265.561462  Topic20  -5.7039  -0.3895
3     경태    1.409146  288.044861  Topic20  -5.6885  -0.4554
68    하소    1.031489  161.507385  Topic20  -6.0005  -0.1888

[1478 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
6917      7  0.832122   가야산
7280      6  0.565936  가을걷이
7680      9  0.660061    가전
3971      1  0.202347    가족
3971      2  0.101173    가족
...     ...       ...   ...
5071      3  0.154556    흑색
5071      6  0.154556    흑색
5071      7  0.154556    흑색
5071     16  0.154556    흑색
5920     14  0.434287    흥원

[3680 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 13, 9, 20, 15, 5, 6, 1, 19, 2, 11, 8, 7, 18, 16, 3, 17, 4, 14, 10])

In [13]:
# 주제 : 폐단

In [14]:
data =[]
for i in range(0,1):
    txt1 = open('문종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '문종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '폐단' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'문종실록'))

In [15]:
# 시리즈 만들기
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("문종실록"," "))
df=df.map(lambda x : x.replace("문종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [16]:
len(df)

219

In [17]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [18]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [19]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	의논	0.010566115379333496
	백성	0.01047614123672247
	하니	0.009341923519968987
	폐단	0.009037605486810207
	수령	0.008930908516049385
	다시	0.008609973825514317
	모두	0.00804784707725048
	감사	0.00791830476373434
	다만	0.007424298673868179
	비록	0.007371628191322088


Topic ID: 1
	하니	0.011695955879986286
	폐단	0.011016049422323704
	백성	0.009200864471495152
	의논	0.00778881274163723
	비록	0.006952198222279549
	모두	0.006920231506228447
	만약	0.006287657655775547
	고을	0.0062837074510753155
	경태	0.005672468803822994
	어찌	0.005153311416506767


Topic ID: 2
	하니	0.009570574387907982
	의논	0.007561186794191599
	모두	0.006577678956091404
	어찌	0.006123355124145746
	다시	0.005742672365158796
	비록	0.005229988135397434
	생각	0.005065188277512789
	반드시	0.004570028278976679
	만약	0.004314135760068893
	수가	0.004217213951051235


Topic ID: 3
	폐단	0.010044027119874954
	하니	0.007983502000570297
	백성	0.007894333451986313
	모두	0.007185791619122028
	의논	0.006766886916011572
	대신	0.006642984692007303
	다른	0.006361155305057764

In [20]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.028330  0.006489       1        1  24.183891
13     0.028631  0.004921       2        1  12.982938
11     0.020280  0.011756       3        1   8.957742
19     0.019767  0.029307       4        1   7.445774
0      0.047525 -0.002126       5        1   5.986314
9      0.005872  0.017273       6        1   5.563701
15    -0.023662  0.007114       7        1   5.169992
18    -0.005814  0.000004       8        1   4.328031
12     0.009175 -0.013083       9        1   4.249372
10    -0.002531 -0.005198      10        1   3.192316
17     0.010028  0.009275      11        1   3.183344
4     -0.010802 -0.012145      12        1   2.495708
6     -0.040793  0.004770      13        1   2.281179
5      0.001844 -0.016951      14        1   2.237352
16    -0.005382 -0.014299      15        1   2.113826
3     -0.039403  0.052378      16        1   2.077696
14     0.023076 -0.024298      17        1   1.271062
1     -0.005475 -0.001795      18        1   0.990706
8     -0.033087 -0.036178      19        1   0.906476
2     -0.027580 -0.017214      20        1   0.382584, topic_info=    Term        Freq       Total Category  logprob  loglift
199   하니  483.000000  483.000000  Default  30.0000  30.0000
471   백성  373.000000  373.000000  Default  29.0000  29.0000
139   폐단  445.000000  445.000000  Default  28.0000  28.0000
105   의논  361.000000  361.000000  Default  27.0000  27.0000
48    모두  374.000000  374.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
8     경태    0.424005  194.151428  Topic20  -5.8943  -0.5607
585   이익    0.356584   74.218826  Topic20  -6.0674   0.2278
180   역사    0.339867   59.333630  Topic20  -6.1154   0.4036
16    군사    0.386884  142.100555  Topic20  -5.9859  -0.3402
139   폐단    0.440945  445.873505  Topic20  -5.8551  -1.3529

[1454 rows x 6 columns], token_table=      Topic      Freq Term
term                      
3861      5  0.497214   가격
1141      1  0.090417  가까이
1141      2  0.090417  가까이
1141      3  0.090417  가까이
1141      4  0.090417  가까이
...     ...       ...  ...
869      12  0.038578   흉년
869      17  0.038578   흉년
5527     13  0.784174   흉풍
5978      2  0.506932   희전
5178      2  0.506815   힘줄

[3245 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 14, 12, 20, 1, 10, 16, 19, 13, 11, 18, 5, 7, 6, 17, 4, 15, 2, 9, 3])